In [3]:
import sqlite3
import pandas as pd
import numpy as np
import plotly.graph_objects as go

In [4]:
conn = sqlite3.connect('../data/checking-logs.sqlite')

In [15]:
query = '''
SELECT uid, numTrials, timestamp
FROM checker
WHERE uid LIKE 'user_%'
      AND status='ready'
      AND labname='project1'
ORDER BY uid
'''
data = pd.io.sql.read_sql(query, conn, parse_dates=['timestamp'])
data.head()

,uid,numTrials,timestamp
0,user_1,1,2020-05-14 20:56:08.898880
1,user_1,2,2020-05-14 20:58:02.313690
2,user_1,3,2020-05-14 20:58:46.322457
3,user_1,4,2020-05-14 21:08:15.434237
4,user_1,5,2020-05-14 21:10:14.867603


In [16]:
data['date'] = data['timestamp'].dt.date
data.drop('timestamp', axis=1, inplace=True)

In [19]:
df = data.groupby(['date', 'uid']).max()['numTrials'].unstack()
df.index = np.arange(len(df.index))
df.iloc[0] = df.iloc[0].fillna(0)
df = df.fillna(method='pad', axis=0)
df = df.transpose()

/tmp/ipykernel_14924/319034841.py:4: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method='pad', axis=0)


In [20]:
initial_data = [go.Scatter(x=np.array([]), y=np.array([]),
                           mode='lines+markers', name=name)
                for name in df.index]

In [21]:
frames = [go.Frame(data=[go.Scatter(
    x=np.arange(i + 1), y=np.array(df.iloc[j, :i + 1]),
    mode='lines+markers', name=df.index[j]
    ) for j in range(len(df.index))]) for i in range(len(df.columns) + 1)]

In [ ]:
fig = go.Figure(
    data=initial_data,
    layout={
        'width': 1200,
        'height': 600,
        'title': 'Dynamic of commits per user in project1',
        'xaxis': {'range': (0, len(df.columns) + 1), 'dtick': 2},
        'yaxis': {'range': (0, round(max(df.max()) + 2, -1) + 1)},
        'updatemenus': [{
            'type': 'buttons',
            'buttons': [{'method': 'animate', 'label': 'play', 'args': [None]}]
            }]}, frames=frames)
fig.show()

In [23]:
conn.close()